# Feature Extraction Pipeline

This notebook contains the steps needed for extracting features from the electrodermal activity, skin temperature and accelerometer data similar to the paper as follows: 

[1] S. Gashi, L. Alecci, E. D. Lascio, M. E. Debus, F. Gasparini and S. Santini, "The Role of Model Personalization for Sleep Stage and Sleep Quality Recognition Using Wearables," in IEEE Pervasive Computing, doi: 10.1109/MPRV.2022.3164334.

If you use snippets of this script, please make sure to cite our paper [1], which is available at: https://ieeexplore.ieee.org/document/9768202 

Before running the feature extraction steps the signals should be preprocessed. We suggest to follow the preprocessing steps of EDArtifact tool presented in https://github.com/shkurtagashi/EDArtifact/blob/master/EDArtifacts_Detection/EDArtifacts_Detection.ipynb

### Import the required packages

Before going further in the data analysis, the required packages for feature extraction should be extracted as follows:

- *fftpack*: to compute fast Fourier transform and extract frequency-domain features. 
- *stats*: to compute statistical features from the time-domain.
- *pywt*: to compute the wavelets.

In [30]:
from collections import Counter
from itertools import chain
import pandas as pd
import numpy as np
import collections
from scipy import fftpack
from scipy.stats import entropy
from scipy import stats
from scipy.stats import linregress
import pywt

## 1. Compute wavelets of the EDA signal

In [1]:
def compute_wavelets(df):
    '''
    This function computes the wavelets of the EDA signal.    
    INPUT:
        df: requires a dataframe with a column named 'EDA_Filtered' 
            
    OUTPUT:
        DF: returns the same dataframe the following columns 'Wavelet1', 'Wavelet2', 'Wavelet3'
    '''

    _, dtw3, dtw2, dtw1 = pywt.wavedec(df['EDA_Filtered'], 'Haar', level=3)
    dtw3_duplicates = list(np.repeat(dtw3, 8))
    dtw2_duplicates = list(np.repeat(dtw2, 4))
    dtw1_duplicates = list(np.repeat(dtw1, 2))

    df['Wavelet3'] = dtw3_duplicates[:len(df)]
    df['Wavelet2'] = dtw2_duplicates[:len(df)]
    df['Wavelet1'] = dtw1_duplicates[:len(df)]
    
    print("1. EDA wavelets computation completed")
    
    return df

## 2. Segment the data

In [32]:
def segment_data(df, segmentation_type):
    '''
    This function segments the signals into 10-minute non-overlapping windows or computes the features over the whole session/night. 
    The sampling frequency of our sensors was set to 4HZ, thereby, the window size for segmenting the data is into 10 minutes
    is 2400 samples (e.g., 10 minutes*4Hz = 2400 data samples).  
    
    INPUT:
        df: requires a dataframe with columns named: 
        'Time', 'EDA_Filtered', 'EDA_Phasic', 'EDA_Tonic', 'TEMP_Filtered',
        'Wavelet3', 'Wavelet2', 'Wavelet1',
       'X_Filtered', 'Y_Filtered', 'Z_Filtered', 'BVP_Filtered', 'User',
       'SessionID', 'SleepQuality_label', 'SleepWake_label',
       'Artifact'
       segmentation_type: refers whether the data should be segmented into 10 minutes windows or by session/night.
        
    OUTPUT:
        df: returns a new dataframe with the following columns
        'EDA_Filtered', 'Time', 'Wavelet1', 'Wavelet2', 'Wavelet3', 'EDA_Phasic', 'EDA_Tonic', 'TEMP_Filtered',
       'X_Filtered', 'Y_Filtered', 'Z_Filtered', 'BVP_Filtered', 'User',
       'SessionID', 'SleepQuality_label', 'SleepWake_label','Artifact'
        Each cell in the dataframe contains an array of 2400 values. 
        
    '''
    
    if segmentation_type == '10min':
        window_size = 2400
        time = df.Time.values.reshape(-1, window_size)
        eda = df.EDA_Filtered.values.reshape(-1, window_size)
        eda_phasic = df.EDA_Phasic.values.reshape(-1, window_size)
        eda_tonic = df.EDA_Tonic.values.reshape(-1, window_size)
        wavelet3 = df.Wavelet3.values.reshape(-1, window_size)
        wavelet2 = df.Wavelet2.values.reshape(-1, window_size)
        wavelet1 = df.Wavelet1.values.reshape(-1, window_size)
        temp = df.TEMP_Filtered.values.reshape(-1, window_size)
        x = df.X_Filtered.values.reshape(-1, window_size)
        y = df.Y_Filtered.values.reshape(-1, window_size)
        z = df.Z_Filtered.values.reshape(-1, window_size)
        bvp = df.BVP_Filtered.values.reshape(-1, window_size)
        user = df.User.values.reshape(-1, window_size)
        sessionID = df.SessionID.values.reshape(-1, window_size)
        sleepQuality = df.SleepQuality_label.values.reshape(-1, window_size)
        sleepWake = df.SleepWake_label.values.reshape(-1, window_size)
        artifact = df.Artifact.values.reshape(-1, window_size)
        psqi = df.PSQI.values.reshape(-1, window_size)
    else:
        time = pd.Series(df.groupby(['User', 'SessionID']).agg({'Time':lambda x: list(x)})['Time'])
        eda = pd.Series(df.groupby(['User', 'SessionID']).agg({'EDA_Filtered':lambda x: list(x)})['EDA_Filtered'])
        eda_phasic = pd.Series(df.groupby(['User', 'SessionID']).agg({'EDA_Phasic':lambda x: list(x)})['EDA_Phasic'])
        eda_tonic = pd.Series(df.groupby(['User', 'SessionID']).agg({'EDA_Tonic':lambda x: list(x)})['EDA_Tonic'])
        wavelet3 = pd.Series(df.groupby(['User', 'SessionID']).agg({'Wavelet3':lambda x: list(x)})['Wavelet3'])
        wavelet2 = pd.Series(df.groupby(['User', 'SessionID']).agg({'Wavelet2':lambda x: list(x)})['Wavelet2'])
        wavelet1 = pd.Series(df.groupby(['User', 'SessionID']).agg({'Wavelet1':lambda x: list(x)})['Wavelet1'])
        temp = pd.Series(df.groupby(['User', 'SessionID']).agg({'TEMP_Filtered':lambda x: list(x)})['TEMP_Filtered'])
        x = pd.Series(df.groupby(['User', 'SessionID']).agg({'X_Filtered':lambda x: list(x)})['X_Filtered'])
        y = pd.Series(df.groupby(['User', 'SessionID']).agg({'Y_Filtered':lambda x: list(x)})['Y_Filtered'])
        z = pd.Series(df.groupby(['User', 'SessionID']).agg({'Z_Filtered':lambda x: list(x)})['Z_Filtered'])
        bvp = pd.Series(df.groupby(['User', 'SessionID']).agg({'BVP_Filtered':lambda x: list(x)})['BVP_Filtered'])
        user = pd.Series(database[['User', 'SessionID']].drop_duplicates()['User'])
        sessionID = pd.Series(database[['User', 'SessionID']].drop_duplicates()['SessionID'])
        sleepQuality = pd.Series(df.groupby(['User', 'SessionID']).agg({'SleepQuality_label':lambda x: list(x)})['SleepQuality_label'])
        sleepWake = pd.Series(df.groupby(['User', 'SessionID']).agg({'SleepWake_label':lambda x: list(x)})['SleepWake_label'])
        artifact = pd.Series(df.groupby(['User', 'SessionID']).agg({'Artifact':lambda x: list(x)})['Artifact'])
        psqi = pd.Series(df.groupby(['User', 'SessionID']).agg({'PSQI':lambda x: list(x)})['PSQI'])


    df = pd.DataFrame(columns=['EDA','Time', 'Wavelet3','Wavelet2','Wavelet1', 'EDA_Phasic', 'EDA_Tonic', 
                               'TEMP_Filtered','X_Filtered', 'Y_Filtered', 'Z_Filtered', 'BVP_Filtered', 'User',
                               'SessionID','SleepQuality_label', 'SleepWake_label', 'Artifact', 'PSQI'])
    df.EDA = pd.Series(list(eda))
    df.EDA_Phasic = pd.Series(list(eda_phasic))
    df.EDA_Tonic = pd.Series(list(eda_tonic))
    df.Time = pd.Series(list(time))   
    df.Wavelet3 = pd.Series(list(wavelet3))   
    df.Wavelet2 = pd.Series(list(wavelet2))   
    df.Wavelet1 = pd.Series(list(wavelet1))  
    df.TEMP_Filtered = pd.Series(list(temp))  
    df.X_Filtered = pd.Series(list(x))  
    df.Y_Filtered = pd.Series(list(y))  
    df.Z_Filtered = pd.Series(list(z))  
    df.BVP_Filtered = pd.Series(list(bvp))  
    df.User = pd.Series(list(user))  
    df.SessionID = pd.Series(list(sessionID))  
    df.SleepQuality_label = pd.Series(list(sleepQuality))  
    df.SleepWake_label = pd.Series(list(sleepWake))  
    df.Artifact = pd.Series(list(artifact))  
    df.PSQI = pd.Series(list(psqi))  

    eda_times = chain.from_iterable(zip(*df.Time.values))
    eda_times = list(eda_times)
    eda_times = eda_times[:len(df)]

    df['Time'] = eda_times
    
    print("2. Data segmentation completed")
        
    return df

## Feature extraction 


We extract features from non-overlapping segments of the signals that could appropriately characterize the sleep-wake and sleep quality events. The three feature groups involve:
- **time-domain**: In the time-domain representation of the sig- nal, we extract statistical features, such as mean, standard deviation, standard error, maximum, mini- mum, median, variance, and quantile with a threshold of 0.7. We also count the number of epochs, storms, and artifacts in each window of the EDA signal and used them as features.
- **frequency-domain**: We transform each signal into the frequency-domain using fast Fourier transform and extract features in this domain. We get the direct current component (dc), the sum of spectral coefficients, the information entropy, and the energy of the signal.
- **time-frequency-domain**: These features include the mean, median, minimum, maximum, quantile, dynamic range, variance, standard error, standard deviation of wavelets coefficients extracted at three different time scales 4, 2, and 1 Hz.

We compute these features for each axis of the ACC sensor, TEMP, and EDA signal, as well as from the phasic, and tonic components the EDA signal. 

## 3. Compute *time-domain* features 

In [42]:
def compute_statistical_features(df):
    '''
    This function computes the time-domain and time-freqency-domain features for each preprocessed 
    component of the provided signals. The features are calculated over the whole window (e.g., 10 min, or whole night). 
    
    INPUT:
        df: requires a dataframe with columns named: 
        'EDA','Wavelet3','Wavelet2','Wavelet1', 'EDA_Phasic', 'EDA_Tonic', 
        'TEMP_Filtered','X_Filtered', 'Y_Filtered', 'Z_Filtered', 'BVP_Filtered'
        
    OUTPUT:
        df: returns the same dataframe with the following features calculated for each column defined in columns variable:
            median:        median value of the 5 second windo
            mean:          average value
            std:           standard deviation
            var:           variance of the signal
            slope:         slope of the signal
            min:           minimum value
            max:           maximum value 
            fdmean:        mean of the first derivative
            fdstd:         standard deviation of the first derivative
            drange:        dynamic range (difference between the maximum and minimum value in the window)
            diffs:         difference between the last and first value in the window
    '''
    
    columns = ['EDA','Wavelet3','Wavelet2','Wavelet1', 'EDA_Phasic', 'EDA_Tonic', 
             'TEMP_Filtered','X_Filtered', 'Y_Filtered', 'Z_Filtered', 'BVP_Filtered']

    for col in columns:
        data = df[col]
        name = col

        medians, means, stds, variances, mins, maxs, fdmeans, sdmeans, fdstds, sdstds, dranges, slopes, diffs  = [], [], [], [], [], [], [], [], [], [], [], [], []

        for i in range(0, len(data)):
            eda = data[i]
            time = range(1, len(eda)+1)

            fd = np.gradient(eda)
            fdmeans.append(np.mean(fd))
            fdstds.append(np.std(fd))
            dranges.append(np.max(eda) - np.min(eda))
            medians.append(np.median(eda))
            means.append(np.mean(eda))
            stds.append(np.std(eda))
            variances.append(np.var(eda))
            mins.append(np.min(eda))
            maxs.append(np.max(eda)) 
            slope, intercept, r_value, p_value, std_err = linregress(time, data[i])  
            slopes.append(slope)
            diffs.append(eda[len(eda)-1] - eda[0])


        df[name+'_median'] = medians
        df[name+'_mean'] = means
        df[name+'_std'] = stds
        df[name+'_var'] = variances
        df[name+'_slope'] = slopes
        df[name+'_min'] = mins
        df[name+'_max'] = maxs
        df[name+'_fdmean'] = fdmeans
        df[name+'_fdstd'] = fdstds
        df[name+'_drange'] = dranges
        df[name+'_diff'] = diffs

        
    print("3.1. Statistical time-domain features extraction completed")
    
    return df

## 4. Compute time-domain features (EDA peaks features)

This part of the code is adapted from the EDA peak detection script available at: https://github.com/MITMediaLabAffectiveComputing/eda-explorer. If you use this part of the code, please cite the following paper:

Taylor, Sara, Natasha Jaques, Weixuan Chen, Szymon Fedor, Akane Sano, and Rosalind Picard. "Automatic identification of artifacts in electrodermal activity data." In 2015 37th Annual International Conference of the IEEE Engineering in Medicine and Biology Society (EMBC), pp. 1934-1937. IEEE, 2015.

In [34]:
SAMPLE_RATE = 4
def findPeaks(data, offset, start_WT, end_WT, thres=0, sampleRate=SAMPLE_RATE):
    '''
        This function finds the peaks of an EDA signal and returns basic properties.
        Also, peak_end is assumed to be no later than the start of the next peak.
        
        ********* INPUTS **********
        data:        DataFrame with EDA as one of the columns and indexed by a datetimeIndex
        offset:      the number of rising samples and falling samples after a peak needed to be counted as a peak
        start_WT:    maximum number of seconds before the apex of a peak that is the ""start"" of the peak
        end_WT:      maximum number of seconds after the apex of a peak that is the ""rec.t/2"" of the peak, 50% of amp
        thres:       the minimum uS change required to register as a peak, defaults as 0 (i.e. all peaks count)
        sampleRate:  number of samples per second, default=8
        
        ********* OUTPUTS **********
        peaks:               list of binary, 1 if apex of SCR
        peak_start:          list of binary, 1 if start of SCR
        peak_start_times:    list of strings, if this index is the apex of an SCR, it contains datetime of start of peak
        peak_end:            list of binary, 1 if rec.t/2 of SCR
        peak_end_times:      list of strings, if this index is the apex of an SCR, it contains datetime of rec.t/2
        amplitude:           list of floats,  value of EDA at apex - value of EDA at start
        max_deriv:           list of floats, max derivative within 1 second of apex of SCR
    '''
    
    EDA_deriv = data['EDA_Phasic'][1:].values - data['EDA_Phasic'][:-1].values
    peaks = np.zeros(len(EDA_deriv))
    peak_sign = np.sign(EDA_deriv)
    for i in range(int(offset), int(len(EDA_deriv) - offset)):
        if peak_sign[i] == 1 and peak_sign[i + 1] < 1:
            peaks[i] = 1
            for j in range(1, int(offset)):
                if peak_sign[i - j] < 1 or peak_sign[i + j] > -1:
                    peaks[i] = 0
                    break

    # Finding start of peaks
    peak_start = np.zeros(len(EDA_deriv))
    peak_start_times = [''] * len(data)
    max_deriv = np.zeros(len(data))
    rise_time = np.zeros(len(data))

    for i in range(0, len(peaks)):
        if peaks[i] == 1:
            temp_start = max(0, i - sampleRate)
            max_deriv[i] = max(EDA_deriv[temp_start:i])
            start_deriv = .01 * max_deriv[i]

            found = False
            find_start = i
            # has to peak within start_WT seconds
            while found == False and find_start > (i - start_WT * sampleRate):
                if EDA_deriv[find_start] < start_deriv:
                    found = True
                    peak_start[find_start] = 1
                    peak_start_times[i] = data.index[find_start]
                    rise_time[i] = get_seconds_and_microseconds(data.index[i] - pd.to_datetime(peak_start_times[i]))

                find_start = find_start - 1

        # If we didn't find a start
            if found == False:
                peak_start[i - start_WT * sampleRate] = 1
                peak_start_times[i] = data.index[i - start_WT * sampleRate]
                rise_time[i] = start_WT

            # Check if amplitude is too small
            if thres > 0 and (data['EDA_Phasic'].iloc[i] - data['EDA_Phasic'][peak_start_times[i]]) < thres:
                peaks[i] = 0
                peak_start[i] = 0
                peak_start_times[i] = ''
                max_deriv[i] = 0
                rise_time[i] = 0

    # Finding the end of the peak, amplitude of peak
    peak_end = np.zeros(len(data))
    peak_end_times = [''] * len(data)
    amplitude = np.zeros(len(data))
    decay_time = np.zeros(len(data))
    half_rise = [''] * len(data)
    SCR_width = np.zeros(len(data))

    for i in range(0, len(peaks)):
        if peaks[i] == 1:
            peak_amp = data['EDA_Phasic'].iloc[i]
            start_amp = data['EDA_Phasic'][peak_start_times[i]]
            amplitude[i] = peak_amp - start_amp

            half_amp = amplitude[i] * .5 + start_amp

            found = False
            find_end = i
            # has to decay within end_WT seconds
            while found == False and find_end < (i + end_WT * sampleRate) and find_end < len(peaks):
                if data['EDA_Phasic'].iloc[find_end] < half_amp:
                    found = True
                    peak_end[find_end] = 1
                    peak_end_times[i] = data.index[find_end]
                    decay_time[i] = get_seconds_and_microseconds(pd.to_datetime(peak_end_times[i]) - data.index[i])

                    # Find width
                    find_rise = i
                    found_rise = False
                    while found_rise == False:
                        if data['EDA_Phasic'].iloc[find_rise] < half_amp:
                            found_rise = True
                            half_rise[i] = data.index[find_rise]
                            SCR_width[i] = get_seconds_and_microseconds(pd.to_datetime(peak_end_times[i]) - data.index[find_rise])
                        find_rise = find_rise - 1

                elif peak_start[find_end] == 1:
                    found = True
                    peak_end[find_end] = 1
                    peak_end_times[i] = data.index[find_end]
                find_end = find_end + 1

            # If we didn't find an end
            if found == False:
                min_index = np.argmin(data['EDA_Phasic'].iloc[i:(i + end_WT * sampleRate)].tolist())
                peak_end[i + min_index] = 1
                peak_end_times[i] = data.index[i + min_index]

    peaks = np.concatenate((peaks, np.array([0])))
    peak_start = np.concatenate((peak_start, np.array([0])))
    max_deriv = max_deriv * sampleRate  # now in change in amplitude over change in time form (uS/second)

    return peaks, peak_start, peak_start_times, peak_end, peak_end_times, amplitude, max_deriv, rise_time, decay_time, SCR_width, half_rise

In [36]:
def get_seconds_and_microseconds(pandas_time):
    return pandas_time.seconds + pandas_time.microseconds * 1e-6

def compute_peaks_features(df):
    '''
    This function computes the peaks features for each 5 second window in the EDA signal.
    
    INPUT:
        df:              requires a dataframe with columns 'EDA_Phasic' and 'Time'
        
    OUTPUT:
        df:              returns the same dataframe with new columnns that contain the following features:
        peaks_p:         number of peaks
        rise_time_p:     average rise time of the peaks
        max_deriv_p:     average value of the maximum derivative
        amp_p:           average amplitute of the peaks
        decay_time_p:    average decay time of the peaks
        SCR_width_p:     average width of the peak (SCR)
        auc_p:           average area under the peak
    '''
    
    thresh =.01
    offset = 1
    start_WT = 3
    end_WT = 10

    data = df.EDA
    times = df.Time

    peaks, rise_times, max_derivs, amps, decay_times, SCR_widths, aucs = [], [], [], [], [], [], []

    for i in range(0, len(data)):
        data_df = pd.DataFrame(columns=["EDA_Phasic", "Time"])
        data_df.EDA_Phasic = pd.Series(list(data[i]))
        data_df.Time = pd.date_range(start=times[i], periods=len(data_df.EDA_Phasic), freq='250ms')
        data_df.set_index(pd.DatetimeIndex(data_df['Time']), inplace=True)

        returnedPeakData = findPeaks(data_df, offset*SAMPLE_RATE, start_WT, end_WT, thresh, SAMPLE_RATE)
        result_df = pd.DataFrame(columns=["peaks","amp","max_deriv","rise_time","decay_time","SCR_width"])
        result_df['peaks'] = returnedPeakData[0]
        result_df['amp'] = returnedPeakData[5]
        result_df['max_deriv'] = returnedPeakData[6]
        result_df['rise_time'] = returnedPeakData[7]
        result_df['decay_time'] = returnedPeakData[8]
        result_df['SCR_width'] = returnedPeakData[9]
        featureData = result_df[result_df.peaks==1][['peaks','rise_time','max_deriv','amp','decay_time','SCR_width']]

        # Replace 0s with NaN, this is where the 50% of the peak was not found, too close to the next peak
        featureData[['SCR_width','decay_time']]=featureData[['SCR_width','decay_time']].replace(0, np.nan)
        featureData['AUC']=featureData['amp']*featureData['SCR_width']

        peaks.append(len(featureData))  
        amps.append(result_df[result_df.peaks != 0.0].amp.mean())
        max_derivs.append(result_df[result_df.peaks != 0.0].max_deriv.mean())
        rise_times.append(result_df[result_df.peaks != 0.0].rise_time.mean())
        decay_times.append(featureData[featureData.peaks != 0.0].decay_time.mean())
        SCR_widths.append(featureData[featureData.peaks != 0.0].SCR_width.mean())
        aucs.append(featureData[featureData.peaks != 0.0].AUC.mean()) 

    df['peaks_p'] = peaks
    df['rise_time_p'] = rise_times
    df['max_deriv_p'] = max_derivs
    df['amp_p'] = amps
    df['decay_time_p'] = decay_times
    df['SCR_width_p'] = SCR_widths
    df['auc_p'] = aucs
    
    print("3.2. Peaks feature extraction completed")
    
    return df


# 4. Compute *frequency-domain* features

In [37]:
def compute_frequency_features(df):
    '''
    This function computes the features from the signal in the frequency-domain.
    
    INPUT:
        df:              requires a dataframe with columns 'EDA', 'EDA_Phasic', 'EDA_Tonic', 
                         'X_Filtered','Y_Filtered','Z_Filtered','TEMP_Filtered'
    OUTPUT:
        df:              returns the same dataframe with new columnns that contain the following features:
        DC:              direct current component    
        CoefSum:         coefficients sum
        DomFreq:         dominant frequency
        Energy:          spectral energy
    '''
        
    cols = ['EDA', 'EDA_Phasic', 'EDA_Tonic', 'X_Filtered','Y_Filtered',
            'Z_Filtered','TEMP_Filtered']

    for c in cols:
        data = df[c]
        name = c

        dc, coeff_sum, dominant_freq, spectral_energy, entropies = [], [], [], [],[]

        f_s = 4  # Sampling rate, or number of measurements per second

        for i in range(0, len(data)):
            eda = data[i]

            X = fftpack.fft(eda)
            freqs = fftpack.fftfreq(len(eda)) * f_s

            ''' Direct Current - DC component '''
            dc.append(abs(np.mean(X) * 32))

            '''Coefficients sum'''
            coeff_sum.append(abs(sum(X)))

            ''' Dominant Frequency '''
            dominant_freq.append(freqs[0])
            index_min = np.argmax(X)
    #         print(index_min)

            ''' Spectral Energy '''
            spectral_energy.append(abs(sum(X))/f_s)

            ''' Information entropy '''
            # Explained in https://stackoverflow.com/questions/30418391/what-is-frequency-domain-entropy-in-fft-result-and-how-to-calculate-it
            # 1. Calculate the PSD of your signal by simply squaring the amplitude spectrum and scaling it by number of frequency bins.
            psd = np.square(X)/f_s

            # 2. Normalize the calculated PSD by dividing it by a total sum.
            norm_psd = psd/sum(psd)

        df[name+'_DC'] = dc
        df[name+'_CoefSum'] = coeff_sum
        df[name+'_DomFreq'] = dominant_freq
        df[name+'_Energy'] = spectral_energy
        
        
    print("3.3. Frequency feature extraction completed")
    
    return df

## Run the feature extraction pipeline

Note that the file should contain the columns as follows: 
            'Time', 'EDA_Filtered', 'EDA_Phasic', 'EDA_Tonic', 'TEMP_Filtered',
            'X_Filtered', 'Y_Filtered', 'Z_Filtered', 'BVP_Filtered', 'User',
            'SessionID', 'SleepQuality_label', 'SleepWake_label','Artifact', 


The 'Artifact' column has been derived using the EDArtifacts tool: 
https://github.com/shkurtagashi/EDArtifact/blob/master/EDArtifacts_Detection/EDArtifacts_Detection.ipynb


In [41]:
def extract_features(file_path):
    for segmentation_type in ['bySession', '10min']:
        for i in ["S01","S02", "S03", "S04", "S05", "S06","S07","S08","S09", "S10","S11", "S12","S13", "S14", "S15","S16"]:
            file_path = "DatasetPath/"+i+".csv"

            # Read data file 
            database = pd.read_csv(file_path)
            database_copy = database.copy()

            # 1. Compute the wavelets
            database_wavelets = compute_wavelets(database_copy)

            if(segmentation_type == '10min'):
                database_wavelets = database_wavelets[:-(len(database_wavelets)%2400)]
            else:
                database_wavelets = database_wavelets[database_wavelets.SleepWake_label == 1]

            # 2. Segment the signals
            database_segmented = segment_data(database_wavelets, segmentation_type)

            # 3. Compute the features
            database_features = compute_statistical_features(database_segmented)
            database_features = compute_peaks_features(database_features)
            database_features.fillna(-1, inplace=True)
            database_features = compute_frequency_features(database_features)
            database_clean.drop(columns=['EDA','Wavelet3','Wavelet2','Wavelet1', 'EDA_Phasic', 'EDA_Tonic', 
                     'TEMP_Filtered','X_Filtered', 'Y_Filtered', 'Z_Filtered', 'BVP_Filtered'], inplace=True)


            # 4. Store the features in a csv file
            if(segmentation_type == '10min'):
                database_clean.to_csv("SleepWake_Data/"+i+"_features.csv", index=False) 
            else:
                database_clean.to_csv("SleepQuality_Data/"+i+"_features.csv", index=False) 


In [ ]:
''' 
    Note that the file should contain the columns as follows: 
                'Time', 'EDA_Filtered', 'EDA_Phasic', 'EDA_Tonic', 'TEMP_Filtered',
                'X_Filtered', 'Y_Filtered', 'Z_Filtered', 'BVP_Filtered', 'User',
                'SessionID', 'SleepQuality_label', 'SleepWake_label','Artifact', 
    
    The 'Artifact' column has been derived using the EDArtifacts tool: 
    https://github.com/shkurtagashi/EDArtifact/blob/master/EDArtifacts_Detection/EDArtifacts_Detection.ipynb
''' 

extract_features(file_path) #TODO: provide the path to your file with preprocessed physiological data